In [2]:
#r "nuget: Shioaji, 1.1.8"

Installed Packages Shioaji, 1.1.8

In [3]:
using System.IO;
using Sinopac.Shioaji;
using System.Text.Json;
Shioaji _api = new Shioaji();

string jsonString = File.ReadAllText(@"D:\Sinopac.json");
JsonElement root = JsonDocument.Parse(jsonString).RootElement;
_api.Login(root.GetProperty("API_Key").GetString(), root.GetProperty("Secret_Key").GetString());

Connecting as shioaji@sinopac on 210.59.255.161:80...


In [4]:
private string GetValidDate(string? AssignDate, bool? Fut)
{
    // 週間休市  只好人工指定前一交易日
    // Kbar用的話，盤後就要T+1，若週五盤後則要T+3，週六凌晨則是T+2，
    if(AssignDate != null) return AssignDate;
    else if (Fut == true)
    {
        DateTime date = DateTime.Now.DayOfWeek switch
        {
            DayOfWeek.Saturday => DateTime.Now.AddDays(2),
            DayOfWeek.Sunday => DateTime.Now.AddDays(1),
            DayOfWeek.Tuesday or DayOfWeek.Wednesday or DayOfWeek.Thursday => (DateTime.Now.Hour >= 15) ? DateTime.Now.AddDays(1): DateTime.Now,
            DayOfWeek.Friday => (DateTime.Now.Hour >= 15) ? DateTime.Now.AddDays(3): DateTime.Now,
            _ => DateTime.Now
        };
        return date.ToString("yyyy-MM-dd");        
    }
    else
    {
        DateTime date = DateTime.Now.DayOfWeek switch
        {
            DayOfWeek.Saturday => DateTime.Now.AddDays(-1),
            DayOfWeek.Sunday => DateTime.Now.AddDays(-2),
            DayOfWeek.Monday => (DateTime.Now.Hour < 9) ? DateTime.Now.AddDays(-3) : DateTime.Now ,
            DayOfWeek.Tuesday => (DateTime.Now.Hour < 9) ? DateTime.Now.AddDays(-1) : DateTime.Now ,
            DayOfWeek.Wednesday => (DateTime.Now.Hour < 9) ? DateTime.Now.AddDays(-1) : DateTime.Now ,
            DayOfWeek.Thursday => (DateTime.Now.Hour < 9) ? DateTime.Now.AddDays(-1) : DateTime.Now ,
            DayOfWeek.Friday => (DateTime.Now.Hour < 9) ? DateTime.Now.AddDays(-1) : DateTime.Now ,
            _ => DateTime.Now
        };
        return date.ToString("yyyy-MM-dd");        
    }
}

private DateTime GetCallbackEndTime()
{
    // 是凌晨就05:00停，若盤中就13:45停止，13:45後(常理來說15:00才會開程式)就隔天05:00停止
    bool preMktTime  = DateTime.Now.TimeOfDay < new TimeSpan( 5,  0, 0);
    bool onMktTime   = DateTime.Now.TimeOfDay > new TimeSpan (8, 45, 0) && DateTime.Now.TimeOfDay < new TimeSpan(13, 45, 0);
    DateTime end = preMktTime ?
                    DateTime.Now.Date.AddHours(5) :
                    (onMktTime ? DateTime.Now.Date.AddHours(13.75) : DateTime.Now.Date.AddHours(29));
    return end;
}

##### 上面三個無腦按

In [ ]:
public Dictionary<string, List<object>> NavBar()
{
    /* 建容器 [key: string] : any[]
        * 前三個裝"指數名稱":[指數、漲跌點、漲跌幅]
        * 後兩個裝裝"項目名稱":[值1、值2]
        */
    Dictionary<string, List<object>> retContainer = new Dictionary<string, List<object>>();

    SJList _IContract = _api.Snapshots(new List<IContract>() {
        _api.Contracts.Futures["TXF"]["TXFR1"],
        _api.Contracts.Indexs["TSE"]["001"],
        _api.Contracts.Indexs["OTC"]["101"]
        });
    foreach (var i in _IContract)
    {
        List<object> _tempIContracts = new List<object>();
        _tempIContracts.Add(i.close);
        _tempIContracts.Add(i.change_price);
        _tempIContracts.Add(i.change_rate);
        retContainer.Add(i.exchange, _tempIContracts);
    }

    var _Margin = _api.Margin();
    List<object> _tempMargins = new List<object>();
    _tempMargins.Add(_Margin.equity_amount);
    _tempMargins.Add(_Margin.available_margin);
    retContainer.Add("Margin", _tempMargins);

    SJList _ListAccounts = _api.ListAccounts();
    List<object> _tempListAccounts = new List<object>();
    _tempListAccounts.Add(_ListAccounts[0].account_id.Substring(4, 3));
    _tempListAccounts.Add(_ListAccounts[1].account_id.Substring(4, 3));
    retContainer.Add("Acct", _tempListAccounts);

    return retContainer;
}

NavBar()

In [ ]:
public Dictionary<DateTime, List<double>> ApexCharts(int LastCount)
{
    /* 建容器 [key: string] : number[] 裝time stamp : [open, high, low, close]
     *
     */
    string bgn = "";
    string end = "";
    // (週二or週三or週四) && (15:00~23:59) ☛ 迄需+1，起就當日
    // (週五) && (15:00~23:59) ☛ 迄需T+3，起就當日
    // (週六) && (00:00~ 04:59) ☛ 迄需T+2，起就T-1
    // else 起迄都當日
    bool cond1 = DateTime.Now.DayOfWeek == DayOfWeek.Tuesday || DateTime.Now.DayOfWeek == DayOfWeek.Wednesday || DateTime.Now.DayOfWeek == DayOfWeek.Thursday;
    bool cond2 = DateTime.Now.TimeOfDay >= new TimeSpan(15, 0, 0) && DateTime.Now.TimeOfDay <= new TimeSpan(23, 59, 59);
    bool cond3 = DateTime.Now.DayOfWeek == DayOfWeek.Friday && DateTime.Now.TimeOfDay >= new TimeSpan(15, 0, 0);
    bool cond4 = DateTime.Now.DayOfWeek == DayOfWeek.Saturday && DateTime.Now.TimeOfDay <= new TimeSpan(23, 59, 59);
    if(cond1 && cond2){
        end = DateTime.Now.AddDays(+1).ToString("yyyy-MM-dd");
        bgn = DateTime.Now.ToString("yyyy-MM-dd");
    }
    else if(cond3){
        end = DateTime.Now.AddDays(+3).ToString("yyyy-MM-dd");
        bgn = DateTime.Now.ToString("yyyy-MM-dd");
    }
    else if(cond4){
        end = DateTime.Now.AddDays(+2).ToString("yyyy-MM-dd");
        bgn = DateTime.Now.AddDays(-1).ToString("yyyy-MM-dd");
    }
    else{
        end = DateTime.Now.ToString("yyyy-MM-dd");
        bgn = end;
    }

    Dictionary<DateTime, List<double>> retKbarsLastCount = new Dictionary<DateTime, List<double>>();
    object _Kbars = _api.Kbars(_api.Contracts.Futures["TXF"]["TXFR1"], bgn, end);
    var res = _Kbars.ToDict();
    long[] ts = res["ts"].ToArray();
    DateTime[] dateTimes = Array.ConvertAll(ts, x => new DateTime(1970, 1, 1, 0, 0, 0, DateTimeKind.Utc).AddTicks(x / 100));
    for (int i = 0; i < dateTimes.Length; i++)
    {
        List<double> _tempKbars = new List<double>();
        _tempKbars.Add(res["Open"][i]);
        _tempKbars.Add(res["High"][i]);
        _tempKbars.Add(res["Low"][i]);
        _tempKbars.Add(res["Close"][i]);
        retKbarsLastCount.Add(dateTimes[i], _tempKbars);
    }
    //若 00:00 ~ 00:59 會取不足60筆
    return retKbarsLastCount.TakeLast(LastCount).ToDictionary(x => x.Key, x => x.Value);
}

ApexCharts(60)

In [ ]:
public Dictionary<string, double> Ticks(int LastCount)
{
    var _TicksQuery = _api.Ticks(_api.Contracts.Futures["TXF"]["TXFR1"], GetValidDate(null, true), TicksQueryType.LastCount, last_cnt: 100);
    Dictionary<string, double> _ret = new Dictionary<string, double>();
    for (int i = 0; i < _TicksQuery.ts.ToArray().Length; i++)
    {
        try { _ret.Add(new DateTime(1970, 1, 1, 0, 0, 0, DateTimeKind.Utc).AddTicks(_TicksQuery.ts[i] / 100).ToString("HH:mm:ss"), _TicksQuery.close[i]); }
        catch (Exception ex) { continue; }
    }
    return _ret.OrderByDescending(x => x.Key).Take(LastCount).ToDictionary(x => x.Key, x => x.Value);
}

In [ ]:
public Dictionary<string, Dictionary<string, double>> Options(string OptionWeek, string yyyyMM, string mode)
{
    double close = _api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
    var OPIContract = new List<IContract>();
    if (mode == "4")
    {
        // BC:[][1] - SC:[][0] + BP:[][1], SP:[][0]
        var strikeLower = Math.Floor(close / 50) * 50;
        var strikeUpper = strikeLower + 100;
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikeUpper + "C"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikeLower + "C"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikeUpper + "P"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikeLower + "P"]);
    }
    else if (mode == "3")
    {
        // SCSP + BP + MTX
        var strikeUpper = Math.Floor(close / 50) * 50 + 100;
        var strikeLower = strikeUpper - 200;
        var strikeBottom = strikeLower - 200;
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikeUpper + "C"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikeLower + "P"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikeBottom + "P"]);
    }
    else if (mode == "9")
    {
        var strikePar = Math.Floor(close / 50) * 50;
        var strike_300 = strikePar - 300;
        var strike_200 = strikePar - 200;
        var strike_100 = strikePar - 100;
        var strike100 = strikePar + 100;
        var strike200 = strikePar + 200;
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike_300 + "P"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike_200 + "C"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike_200 + "P"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike_100 + "C"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike_100 + "P"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikePar + "C"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strikePar + "P"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike100 + "C"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike100 + "P"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike200 + "C"]);
        OPIContract.Add(_api.Contracts.Options[OptionWeek][OptionWeek + yyyyMM + strike200 + "P"]);
    }

    var ret = _api.Snapshots(OPIContract);
    var _ret = new Dictionary<string, Dictionary<string, double>>();
    foreach (var item in ret)
    {
        var code = item.code.Substring(3, 5) + (item.code[8] < 'L' ? "C" : "P");
        if (!_ret.ContainsKey(code)) _ret[code] = new Dictionary<string, double>();
        _ret[code]["buy_price"] = item.buy_price;
        _ret[code]["sell_price"] = item.sell_price;
    }
    return _ret;
}
Options("TX1", "202403", "9")

In [ ]:
public Dictionary<string, Dictionary<string, object>> GetPositions()
{
    var res = _api.ListPositions(_api.FutureAccount);

    var _ret = new Dictionary<string, Dictionary<string, object>>();
    for (int i = 0; i < res.Count; i++)
    {
        {
            var code = res[i].code;
            _ret[code] = new Dictionary<string, object>();
            _ret[code]["direction"] = res[i].direction;
            _ret[code]["quantity"] = res[i].quantity;
            _ret[code]["price"] = res[i].price;
            _ret[code]["last_price"] = res[i].last_price;
            _ret[code]["pnl"] = res[i].pnl;
        }
    }
    return _ret;
}
GetPositions()

In [ ]:
public Dictionary<string, Dictionary<string, object>> StocksTable(string mode)
{
    List<string> PctRank = _api.Scanners(scannerType: ScannerType.ChangePercentRank, date: GetValidDate(null, null), count: 75)
    .Select(x => (string)x.code).ToList();
    List<string> AmtRank = _api.Scanners(scannerType: ScannerType.AmountRank, date: GetValidDate(null, null), count: 75)
    .Select(x => (string)x.code).ToList();
    var IntersectRank = PctRank.Intersect(AmtRank).ToList();

    List<string> TW30 = new List<string> {
        "2330", "2454", "2317", "2382", "2412", "2881", "2308", "6505", "2882", "2303",
        "3711", "2891", "2886", "1303", "1301", "1216", "2002", "2884", "6669", "5880",
        "3045", "2892", "2207", "2885", "3008", "3231", "1326", "2357", "2603", "3034",
    };
    List<string> Watchlist = IntersectRank.Except(TW30).ToList();

    var _AmtRank = _api.Scanners(scannerType: ScannerType.AmountRank, date: GetValidDate(null, null), count: 75);
    var _ret = new Dictionary<string, Dictionary<string, object>>();
    for (int i = 0; i < _AmtRank.Count; i++)
    {
        if ((mode == "TW30" ? TW30 : Watchlist).Contains(_AmtRank[i].code))
        {
            var name = _AmtRank[i].name;
            _ret[name] = new Dictionary<string, object>();
            _ret[name]["code"] = _AmtRank[i].code;
            _ret[name]["close"] = _AmtRank[i].close;
            _ret[name]["price_range"] = _AmtRank[i].price_range;
            _ret[name]["tick_type"] = _AmtRank[i].tick_type;
            _ret[name]["change_price"] = _AmtRank[i].change_price;
            _ret[name]["average_price"] = _AmtRank[i].average_price;
            _ret[name]["rank_value"] = Math.Round(_AmtRank[i].rank_value / 100_000_000, 2);
            _ret[name]["buy_price"] = _AmtRank[i].buy_price;
            _ret[name]["sell_price"] = _AmtRank[i].sell_price;
        }
    }
    var __ret = _ret.Where(x => Convert.ToDouble(x.Value["rank_value"]) >= 5);
    return __ret.Take(15).ToDictionary(x => x.Key, x => x.Value);
}
StocksTable("").Keys

In [ ]:
public Dictionary<string, List<double>> ApexChartsOHLC(string target)
{
    /*
    *選股範圍 電金非金電 vs 漲幅排行，IContract兩種切換
    *轉成{"2330":{"oepn" = 111, "high" = 222, "low" = 333, "close" = 444}, "2454":{"oepn" = 111, "high" = 222, "low" = 333, "close" = 444}}
    *linq過篩後再次轉回 Dictionary<string, Dictionary<string, object>>
    */
    List<string> TENN = new List<string> { "2330", "2454", "2317", "2382", "2308", "2303", "3711", "6669", "3008", "3231" };
    List<string> TFNN = new List<string> { "2881", "2882", "2891", "2886", "2884", "5880", "2892", "2885", "2880", "2890" };
    List<string> XINN = new List<string> { "6505", "1303", "1301", "1216", "2002", "2207", "1326", "2603", "5871", "2912" };
    List<string> ScannersAmountRank = _api.Scanners(scannerType: ScannerType.ChangePercentRank, date: GetValidDate(null, null), count: 30)
    .Select(x => (string)x.code).ToList();

    var SectorComponentsIContract = new List<IContract>();
    if (target == "TENN")
    {
        foreach (string code in TENN) SectorComponentsIContract.Add(_api.Contracts.Stocks["TSE"][code]);
    }
    else if (target == "TFNN")
    {
        foreach (string code in TFNN) SectorComponentsIContract.Add(_api.Contracts.Stocks["TSE"][code]);
    }
    else if (target == "XINN")
    {
        foreach (string code in XINN) SectorComponentsIContract.Add(_api.Contracts.Stocks["TSE"][code]);
    }
    else
    {
        foreach (string code in ScannersAmountRank)
        {
            Exchange exchange = _api.Contracts.Stocks["TSE"].ContainsKey(code) ? Exchange.TSE : Exchange.OTC;
            SectorComponentsIContract.Add(_api.Contracts.Stocks[exchange.ToString()][code]);
        };
    }

    Dictionary<string, List<double>> _ret = new Dictionary<string, List<double>>();
    var _Snapshots = _api.Snapshots(SectorComponentsIContract);
    for (int i = 0; i < _Snapshots.ToArray().Length; i++)
    {
        List<double> _tempSnapshots = new List<double>();
        var _lastClose = _Snapshots[i].close - _Snapshots[i].change_price;
        _tempSnapshots.Add(Math.Round(100 * (_Snapshots[i].open - _lastClose) / _lastClose, 2));
        _tempSnapshots.Add(Math.Round(100 * (_Snapshots[i].high - _lastClose) / _lastClose, 2));
        _tempSnapshots.Add(Math.Round(100 * (_Snapshots[i].low - _lastClose) / _lastClose, 2));
        _tempSnapshots.Add(Math.Round(100 * (_Snapshots[i].close - _lastClose) / _lastClose, 2));
        _tempSnapshots.Add(Math.Round(_Snapshots[i].total_amount / 100_000_000d, 0));
        _ret.Add(_Snapshots[i].code, _tempSnapshots);
    }
    var test = _ret.Where(x => x.Value[4] >= 1).Take(15);
    return test.ToDictionary(x => x.Key, x => x.Value);
}
ApexChartsOHLC("")

# Order

In [ ]:
public Sinopac.Shioaji.Trade MXFPlaceOrder(string _action, int _price, string _octype, bool _log)
{
    var _contract = _api.Contracts.Futures["MXF"]["MXF202403"];
    var _futOptOrder = new FutOptOrder()
    {
        action = _action == "Buy" ? "Buy" : "Sell",
        price = _price,
        quantity = 1,
        price_type = "LMT",
        order_type = "ROD",
        octype = _octype == "DayTrade" ? "DayTrade" : "Auto" , //'Auto', 'New', 'Cover', 'DayTrade'"
    };
    var _trade = _api.PlaceOrder(_contract, _futOptOrder);

    if (_log)
    {
        var ret = new
        {
            // 分contract, order, status。要把原廠Log改成false不然很煩
            status = _trade.status.status,
            price = _trade.order.price,
            deal_quantity = _trade.status.deal_quantity,
            cancel_quantity = _trade.status.cancel_quantity,
            action = _trade.order.action,
            //order_lot = _trade.order.order_lot,
            //quantity = _trade.order.quantity,
            //seqno = _trade.order.seqno,  // 234925
        };
        Console.WriteLine("委託細節" + string.Join(", ", ret));
    }
    return _trade;
}

// Sinopac.Shioaji.Trade _trade = MXFPlaceOrder("Sell", 19467, "Auto", true);
// _api.CancelOrder(_trade)
Sinopac.Shioaji.Trade _trade_ = MXFPlaceOrder("Buy", 19400, "Auto", true);
// _api.CancelOrder(_trade_)
_api.UpdateOrder(_trade_)

In [ ]:
using System.Threading;

public Sinopac.Shioaji.Trade MXFPlaceOrder(string _action, double _price, string _octype, bool _log)
{
    var _contract = _api.Contracts.Futures["MXF"]["MXF202403"];
    var _futOptOrder = new FutOptOrder()
    {
        action = _action == "Buy" ? "Buy" : "Sell",
        price = _price,
        quantity = 1,
        price_type = "LMT",
        order_type = "ROD",
        octype = _octype == "DayTrade" ? "DayTrade" : "Auto" , //'Auto', 'New', 'Cover', 'DayTrade'"
    };
    var _trade = _api.PlaceOrder(_contract, _futOptOrder);

    if (_log)
    {
        var ret = new
        {
            // 分contract, order, status。要把原廠Log改成false不然很煩
            status = _trade.status.status,
            price = _trade.order.price,
            deal_quantity = _trade.status.deal_quantity,
            cancel_quantity = _trade.status.cancel_quantity,
            action = _trade.order.action,
            //order_lot = _trade.order.order_lot,
            //quantity = _trade.order.quantity,
            //seqno = _trade.order.seqno,  // 234925
        };
        Console.WriteLine("委託細節" + string.Join(", ", ret));
    }
    return _trade;
}

// 查成本價、建容器、創「int安全邊際」&「bool拉開就調成保本」
List<FuturePosition> _src = _api.ListPositions(_api.FutureAccount);
double EntryPrice = (double)_src.Where(x => x.code.Contains("MXF")).Select(x => x.price).FirstOrDefault();

List<double> SwingHighs = new List<double>();
_api.Subscribe(_api.Contracts.Futures["TXF"]["TXFR1"], QuoteType.tick, version: QuoteVersion.v1);

int margin = 10;
double new_EntryPrice = 0;

while (DateTime.Now < GetCallbackEndTime())
{
    if((SwingHighs.Last() > EntryPrice + margin) && new_EntryPrice == 0) // 若拉開就保單
    {
        new_EntryPrice = EntryPrice + 2.0;
    }
    else if(SwingHighs.Last() < EntryPrice - margin)  // 口憐吃筍
    {
        Sinopac.Shioaji.Trade _trade_ = MXFPlaceOrder("Sell", SwingHighs.Last(), "Auto", true);
    }
    else if (new_EntryPrice != 0)  // 拉開已調成保本，自高點拉回就移停 or 打保本
    {
        Sinopac.Shioaji.Trade _trade_ = (SwingHighs.Last() < (SwingHighs.Max() - margin)) ? 
        MXFPlaceOrder("Sell", SwingHighs.Last(), "Auto", true) : null;
    }
    else  // 持續監聽
    {
        Console.WriteLine($"夾在{EntryPrice-10}~{SwingHighs.Max()}死撐中");
    }
    Thread.Sleep(30_000);
}

# 舊

In [ ]:
#region History profit and loss log. 歷史損益API雖設計支援日期區間，但現行日期區間多日會報錯entry_price不可為{null}
public void Pnl(string bgn, string end)
{
    try
    {
        var src = _api.ListProfitLossSummary(bgn, end, _api.FutureAccount).profitloss_summary[0];
        var _Pnl = new
        {
            code = (src.code.Substring(3, 5)) + (src.code[8] < 'L' ? "Call" : "Put"),
            direction = src.direction,
            entry_price = src.entry_price,
            cover_price = src.cover_price,
            netPnl = src.pnl - src.fee - src.tax
        };
        Console.WriteLine(string.Join("\n", _Pnl));
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Failed {ex.Message}");
    }
}
#endregion

In [ ]:
public IEnumerable<KeyValuePair<string, Dictionary<string, object>>> Snapshots_TW30(string target)
{
    /*
    *選股範圍 TW30 vs 漲幅排行，IContract兩種切換
    *轉成{"2330":{"oepn" = 111, "high" = 222, "low" = 333, "close" = 444}, "2454":{"oepn" = 111, "high" = 222, "low" = 333, "close" = 444}}
    *linq過篩後再次轉回 Dictionary<string, Dictionary<string, object>>
    */
    List<string> Sids30 = new List<string> {
        "2330", "2454", "2317", "2382", "2412", "2881", "2308", "6505", "2882", "2303",
        "3711", "2891", "2886", "1303", "1301", "1216", "2002", "2884", "6669", "5880",
        "3045", "2892", "2207", "2885", "3008", "3231", "1326", "2357", "2603", "3034",
    };
    List<string> Sids20 = _api.Scanners(scannerType: ScannerType.ChangePercentRank, date: GetValidDate(null, null), count: 30)
    .Select(x => (string)x.code).ToList();
    
    var IContract50 = new List<IContract>();
    if (target == "") foreach (string code in Sids30) IContract50.Add(_api.Contracts.Stocks["TSE"][code]);
    else
    {
        foreach (string code in Sids20)
        {
            Exchange exchange = _api.Contracts.Stocks["TSE"].ContainsKey(code) ? Exchange.TSE : Exchange.OTC;
            IContract50.Add(_api.Contracts.Stocks[exchange.ToString()][code]);
        };
    }

    var _Snapshots = _api.Snapshots(IContract50);
    var _ret = new Dictionary<string, Dictionary<string, object>>();
    foreach (var item in _Snapshots)
    {
        var code = item.code;
        if (!_ret.ContainsKey(code)) _ret[code] = new Dictionary<string, object>();
        _ret[code]["code"] = item.code;
        _ret[code]["ts"] = new DateTime(1970, 1, 1, 0, 0, 0, DateTimeKind.Utc).AddTicks(item.ts / 100).ToString("yyyy-MM-dd");
        _ret[code]["open"] = item.open;
        _ret[code]["high"] = item.high;
        _ret[code]["low"] = item.low;
        _ret[code]["close"] = item.close;
        _ret[code]["tick_type"] = item.tick_type;
        _ret[code]["change_price"] = item.change_price;
        _ret[code]["change_rate"] = item.change_rate;
        _ret[code]["change_type"] = item.change_type;
        _ret[code]["average_price"] = item.average_price;
        _ret[code]["volume"] = item.volume;
        _ret[code]["total_amount"] = item.total_amount;
        _ret[code]["buy_price"] = item.buy_price;
        _ret[code]["sell_price"] = item.sell_price;
    }
    IEnumerable<KeyValuePair<string, Dictionary<string, object>>> test = _ret
    .Where(x => Convert.ToInt64(x.Value["total_amount"]) >= 500_000_00 
    & Convert.ToInt64(x.Value["close"]) > 30);
    return test;
}

Snapshots_TW30("")

In [ ]:
            /*建容器 [key: string] : number[] 裝time stamp : 逐筆成交價
             */
            //Dictionary<string, double> retTicksLastCount = new Dictionary<string, double>();

            //string end = "";
            //if (DateTime.Now.TimeOfDay >= new TimeSpan(15, 0, 0) && DateTime.Now.TimeOfDay <= new TimeSpan(23, 59, 59))
            //{ end = DateTime.Now.AddDays(+1).ToString("yyyy-MM-dd"); }
            //else { end = DateTime.Now.ToString("yyyy-MM-dd"); }

            //Ticks _TicksQuery = _api.Ticks(_api.Contracts.Futures["TXF"]["TXFR1"], end, TicksQueryType.LastCount, last_cnt: LastCount);
            //List<DateTimeOffset> generalTimes = new List<DateTimeOffset>();
            //foreach (long timestamp in _TicksQuery.ts)
            //{
            //    DateTime dateTime = DateTimeOffset.FromUnixTimeMilliseconds(timestamp / 1000000).UtcDateTime;
            //    generalTimes.Add(new DateTimeOffset(dateTime, TimeSpan.Zero));
            //}
            //List<string> tsFormat = new List<string>();
            //foreach (var i in generalTimes) tsFormat.Add(i.ToString("HH:mm:ss"));

            //for (int i = 0; i < tsFormat.ToArray().Length; i++)
            //{
            //    if (retTicksLastCount.ContainsKey(tsFormat[i])) { retTicksLastCount[tsFormat[i]] = _TicksQuery.close[i]; }
            //    else { retTicksLastCount.Add(tsFormat[i], _TicksQuery.close[i]); }
            //}
            //return retTicksLastCount.OrderByDescending(x => x.Key).ToDictionary(x => x.Key, x => x.Value);

In [ ]:
public Dictionary<string, List<object>> SectorsolumnsTable()
{
    // Scanners漲幅榜用linq篩出 $20且過五億的標的 把Code存 List<string>
    List<string> ls_sids = new List<string>();
    //foreach (var i in _api.Scanners(scannerType: ScannerType.ChangePercentRank, date: DateTime.Now.ToString("yyyy-MM-dd"), count: 50)
    foreach (var i in _api.Scanners(scannerType: ScannerType.ChangePercentRank, date: "2024-02-20", count: 50)
    .Where(x => x.close > 20 && x.total_amount > 500_000_000)) { ls_sids.Add(i.code); }


    // 從openapi抓 List<string> 股票們存成 Dictionary<string, List<object>> ☛ { 股號: [股名, 月均價]}
    var client = new HttpClient();
    var response = client.GetAsync("https://openapi.twse.com.tw/v1/" + "exchangeReport/STOCK_DAY_AVG_ALL").Result; // 上市個股日收盤價及月平均價
    var json = response.Content.ReadAsStringAsync().Result;
    List<dynamic> src = JsonConvert.DeserializeObject<List<dynamic>>(json);
    Dictionary<string, List<object>> retDict = new Dictionary<string, List<object>>();
    foreach (var i in src.Where(x => ls_sids.Contains(x.Code.ToString())))
    {
        List<object> _retDict = new List<object>();
        _retDict.Add(i.Name.ToString());
        _retDict.Add((Double)i.MonthlyAveragePrice);
        retDict.Add(i.Code.ToString(), _retDict);
    }


    // 同格式的Dictionary<string, List<object>> 另用Snapshots對同群標的做 { 股號: [即時價, 漲額, 成交值億]}
    var obj_IContract = new List<IContract>();
    foreach (var i in retDict.Keys)
        try { obj_IContract.Add(_api.Contracts.Stocks["TSE"][i]); }
        catch (Exception ex)
        {
            Console.WriteLine($"宣告{ex}但沒用到會報alert");
            obj_IContract.Add(_api.Contracts.Stocks["OTC"][i]);
        }

    Dictionary<string, List<object>> retDict1 = new Dictionary<string, List<object>>();
    foreach (var i in _api.Snapshots(obj_IContract))
    {
        List<object> _retDict1 = new List<object>();
        _retDict1.Add(i.close);
        _retDict1.Add(i.change_rate);
        _retDict1.Add(Math.Round(i.total_amount / 100_000_000d, 2));
        retDict1.Add(i.code, _retDict1);
    }


    // 把兩個 Dictionary<string, List<object>>併起來，再補上乖離 ☛ { 股號:         [股名, 月均價, 即時價, 漲幅, 成交值億, 距月均價多遠]}
    // 這個Concat的dict留到react定義 interface                  ☛  [key: string]: [string, number, number, number, number, number];
    var ret = retDict.Concat(retDict1).GroupBy(kvp => kvp.Key).ToDictionary(g => g.Key, g => g.SelectMany(kvp => kvp.Value).ToList());
    foreach (var i in ret.Keys)
    {
        ret[i].Add(Math.Round(
            ((double)ret[i][2] / (double)ret[i][1]) - 1
            , 2));
    }

    // 篩漲超過月線x%的
    return ret.Where(x => (Double)x.Value[5] >= 0.00).ToDictionary(x => x.Key, x => x.Value);
}



In [ ]:
public IEnumerable<KeyValuePair<string, Dictionary<string, object>>> ScannersChangePercentRank(int TopN)
// public dynamic ScannersChangePercentRank(int TopN)
{
    /* 建容器 [key: string] : number[] 裝"股名":[開盤漲幅、最高漲幅、最低漲幅、當前漲幅、累計成交值]*/

    var _ScannersAmountRank = _api.Scanners(scannerType: ScannerType.ChangePercentRank, date: GetValidDate(), count: TopN)
    .Select(x => (string)x.code).ToList();

    var SectorComponentsIContract = new List<IContract>();
    foreach (string code in _ScannersAmountRank)
    {
        Exchange exchange = _api.Contracts.Stocks["TSE"].ContainsKey(code) ? Exchange.TSE : Exchange.OTC;
        SectorComponentsIContract.Add(_api.Contracts.Stocks[exchange.ToString()][code]);
    };

    var _Snapshots = _api.Snapshots(SectorComponentsIContract);

    var _ret = new Dictionary<string, Dictionary<string, object>>();
    // for (int i = 0; i < _Snapshots.ToArray().Length; i++)
    foreach (var item in _Snapshots)
    {
        var code = item.code;
        if (!_ret.ContainsKey(code)) _ret[code] = new Dictionary<string, object>();
        _ret[code]["code"] = item.code;
        _ret[code]["ts"] = new DateTime(1970, 1, 1, 0, 0, 0, DateTimeKind.Utc).AddTicks(item.ts / 100).ToString("yyyy-MM-dd");
        _ret[code]["open"] = item.open;
        _ret[code]["high"] = item.high;
        _ret[code]["low"] = item.low;
        _ret[code]["close"] = item.close;
        _ret[code]["tick_type"] = item.tick_type;
        _ret[code]["change_price"] = item.change_price;
        _ret[code]["change_rate"] = item.change_rate;
        _ret[code]["change_type"] = item.change_type;
        _ret[code]["average_price"] = item.average_price;
        _ret[code]["volume"] = item.volume;
        _ret[code]["total_amount"] = item.total_amount;
        _ret[code]["buy_price"] = item.buy_price;
        _ret[code]["sell_price"] = item.sell_price;
    }
    IEnumerable<KeyValuePair<string, Dictionary<string, object>>> test = _ret.Where(x => Convert.ToInt64(x.Value["total_amount"]) >= 500_000_00 & Convert.ToInt64(x.Value["close"]) > 30);
    return test;
}

ScannersChangePercentRank(10)

foreach (var entry in ScannersChangePercentRank(10))
{
    Console.WriteLine($"'{entry.Key}':[");
    foreach (var value in entry.Value)
    {
        Console.WriteLine($"{value},");
    }
    Console.WriteLine("]");
}

In [ ]:
// public Dictionary<string, double> TicksLastCount(int LastCount)
// {
//     /*建容器 [key: string] : number[] 裝time stamp : 逐筆成交價
//         */
//     Dictionary<string, double> retTicksLastCount = new Dictionary<string, double>();

//     string end = "";
//     if (DateTime.Now.TimeOfDay >= new TimeSpan(15, 0, 0) && DateTime.Now.TimeOfDay <= new TimeSpan(23, 59, 59))
//     { end = DateTime.Now.AddDays(+1).ToString("yyyy-MM-dd"); }
//     else { end = DateTime.Now.ToString("yyyy-MM-dd"); }
//     Ticks _TicksQuery = _api.Ticks(_api.Contracts.Futures["TXF"]["TXFR1"], end, TicksQueryType.LastCount, last_cnt: LastCount);
//     List<DateTimeOffset> generalTimes = new List<DateTimeOffset>();
//     foreach (long timestamp in _TicksQuery.ts)
//     {
//         DateTime dateTime = DateTimeOffset.FromUnixTimeMilliseconds(timestamp / 1000000).UtcDateTime;
//         generalTimes.Add(new DateTimeOffset(dateTime, TimeSpan.Zero));
//     }
//     List<string> tsFormat = new List<string>();
//     foreach (var i in generalTimes) tsFormat.Add(i.ToString("HH:mm:ss"));

//     for (int i = 0; i < tsFormat.ToArray().Length; i++)
//     {
//         if (retTicksLastCount.ContainsKey(tsFormat[i])) { retTicksLastCount[tsFormat[i]] = _TicksQuery.close[i]; }
//         else { retTicksLastCount.Add(tsFormat[i], _TicksQuery.close[i]); }
//     }
//     return retTicksLastCount.OrderByDescending(x => x.Key).ToDictionary(x => x.Key, x => x.Value);
// }
// TicksLastCount(10)